In [2]:
import pandas as pd

# Importo funciones del directorio src
# from src.toolbox import test_toolbox
import src.limpieza as l

# Cargo el dataframe
df = pd.read_csv("input/covid_19_containment_measures_data.csv")

# Aplico las funciones para limpiar datos
drop_nan_by_field
normalize_us_data
divide_csv_field
df = l.df_select_columns(df, "Country", "Date Start", "Date end intended", "Keywords")

normalize_keyword_field


# Al final me quedo con al dataframe limpio.

print("Setup completado!")

Setup completado!
